In [1]:
import numpy as np, pandas as pd 
import matplotlib.pyplot as plt
import torch
import time 
import random

In [2]:
initial_grid = np.array([[None, 4, None, None, None, None, None],
 [None, None, 6, 3, None, None, 6],
 [None, None, None, None, None, 5, 5],
 [None, None, None, 4, None, None, None],
 [4, 7, None, None, None, None, None],
 [2, None, None, 7, 4, None, None],
 [None, None, None, None, None, 1, None]])

In [3]:
solution_grid = np.array([[7, 4, 3, None, 6, None, None],
 [None, None, 6, 3, 5, None, 6],
 [None, None, 5, None, 5, 5, 5],
 [None, 3, 6, 4, None, None, 7],
 [4, 7, None, None, None, 7, 2],
 [2, None, None, 7, 4, 7, None],
 [7, 6, None, 6, None, 1, None]])

In [4]:
almost_solution_grid = np.array([[7, 4, 3, None, 6, None, None],
 [None, None, 6, 3, 5, None, 6],
 [None, None, 5, None, 5, 5, 5],
 [None, 3, 6, 4, None, None, None],
 [4, 7, None, None, None, None, None],
 [2, None, None, 7, 4, None, None],
 [7, 6, None, 6, None, 1, None]])

In [5]:
bad_solution_grid = np.array([[7, 4, 3, 7, 6, None, None],
 [None, None, 6, 3, 5, None, 6],
 [None, None, 5, None, 5, 5, 5],
 [None, 3, 6, 4, None, None, 7],
 [4, 7, None, None, None, 7, 2],
 [2, None, None, 7, 4, 7, None],
 [7, 6, None, 6, None, 1, None]])

In [6]:
unconnected_solution = np.array([[7, 4, 3, None, 6, None, None],
 [None, None, 6, 3, 5, None, 6],
 [None, None, 5, None, 5, 5, 5],
 [None, 3, 6, 4, None, 7, None],
 [4, 7, None, None, None, 7, 2],
 [2, None, None, 7, 4, None, 7],
 [7, 6, None, 6, None, 1, None]])

In [7]:
# fill Nones with 0
def fill_nones(grid):
    return np.where(grid == None, 0, grid)

In [8]:
# make remaining_numbers dict
def make_remaining_numbers(grid):
    remaining_numbers = {7: 7, 6: 6, 5: 5, 4: 4, 3: 3, 2: 2, 1: 1}
    for i in range(7):
        for j in range(7):
            if grid[i, j] != None:
                remaining_numbers[grid[i, j]] -= 1
    # value of 0 should be sum of all nones
    remaining_numbers[0] = 49 - np.count_nonzero(grid)
    return remaining_numbers

In [9]:
# function to check subsquares
def check_sub_squares(grid):
    # every 2-by-2 subsquare in the completed grid must contain at least one empty cell.
    # fill None with 0
    grid = np.where(grid == None, 0, grid).copy()
    for i in range(0, 7-2):
        for j in range(0, 7-2):
            sub_square = grid[i:i+2, j:j+2]
            # need to check there are exactly 3 non zero cells
            non_zero_count = np.count_nonzero(sub_square)
            if non_zero_count == 4:
                #print('found false sub square')
                #print(sub_square)
                return False
    return True

In [10]:
# function to check sum of rows and cols is 20
def check_row_col(grid):
    # replace None with 0, temporarily
    grid = np.where(grid == None, 0, grid).copy()
    # check rows
    for row in grid:
        if np.sum(row) != 20:
            return False
    # check cols
    for col in grid.T:
        if np.sum(col) != 20:
            return False
    return True

In [11]:
# checking the solution is connected
def to_ones(grid):
    output = np.zeros((7, 7))
    for i in range(7):
        for j in range(7):
            if grid[i, j] != None and grid[i, j] != 0:
                output[i, j] = 1
    return output

from collections import deque

# breadth first search
def bfs(grid, i, j):
    queue = deque([(i, j)])
    while queue:
        i, j = queue.popleft()
        if grid[i][j] == 1:
            grid[i][j] = -1
            if i > 0:
                queue.append((i - 1, j))
            if i < len(grid) - 1:
                queue.append((i + 1, j))
            if j > 0:
                queue.append((i, j - 1))
            if j < len(grid[0]) - 1:
                queue.append((i, j + 1))

def count_connected_components(grid):
    count = 0
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j] == 1:
                count += 1
                bfs(grid, i, j)
    return count

def is_connected(grid):
    return count_connected_components(grid) == 1

In [12]:
# lets make a recursive function to fill the grid, it will use remaining_numbers dict to keep track of numbers to fill
time_last_print = time.time()
def fill_grid(grid):
    global remaining_numbers
    global time_last_print
    # check if grid is filled

    if check_sub_squares(grid) and check_row_col(grid):
        print('found solution')
        print(fill_nones(grid))
        print('connected:', is_connected(to_ones(grid)))
        print('---------------------------------')
        return

    if np.all(grid != None):
        # check subsquares
        if not check_sub_squares(grid):
            return
        
            

    # check if row and col sums are 20
    # replace None with 0, temporarily
    grid_temp = np.where(grid == None, 0, grid).copy()
    for i in range(7):
        row_sum = np.sum(grid_temp[i, :])
        if row_sum > 20:
            return
        col_sum = np.sum(grid_temp[:, i])
        if col_sum > 20:
            return
    # print if more than 2 seconds elapsed
    if time.time() - time_last_print > 2:
        print('time elapsed')
        print(grid)
        print(time.time() - time_last_print)
        time_last_print = time.time()
    # find first empty cell
    # print(grid)
    for i in range(7):
        for j in range(7):
            if grid[i, j] == None:
                # fill cell with each number in remaining_numbers
                for num in remaining_numbers:
                    if remaining_numbers[num] > 0:
                        # make a copy of grid and remaining_numbers
                        grid_copy = grid.copy()
                        # fill cell with num
                        grid_copy[i, j] = num
                        # decrement remaining_numbers
                        remaining_numbers[num] -= 1
                        # call fill_grid recursively
                        fill_grid(grid_copy)
                        remaining_numbers[num] += 1
                return 
remaining_numbers = make_remaining_numbers(almost_solution_grid)
print(remaining_numbers)
grid = almost_solution_grid.copy()
fill_grid(grid)

{7: 3, 6: 0, 5: 0, 4: 0, 3: 0, 2: 1, 1: 0, 0: 25}
found solution
[[7 4 3 0 6 0 0]
 [0 0 6 3 5 0 6]
 [0 0 5 0 5 5 5]
 [0 3 6 4 0 7 0]
 [4 7 0 0 0 7 2]
 [2 0 0 7 4 0 7]
 [7 6 0 6 0 1 0]]
connected: False
---------------------------------
found solution
[[7 4 3 0 6 0 0]
 [0 0 6 3 5 0 6]
 [0 0 5 0 5 5 5]
 [0 3 6 4 0 0 7]
 [4 7 0 0 0 7 2]
 [2 0 0 7 4 7 0]
 [7 6 0 6 0 1 0]]
connected: True
---------------------------------
